# An Introduction to Optimisation via Simulation
## A tutorial for the Simulation Workshop 2020
By Christine S.M Currie and Tom Monks

In [1]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [2]:
from ovs.toy_models import (custom_guassian_model,
                            guassian_sequence_model,
                            random_guassian_model,
                            ManualOptimiser)

This tutorial will be making use of the following models.  

### Creating a simple sequence of normal distributions

A simple model can be created using the `guassian_sequance_model()` function.  This creates a simulation model where the output of each design follows a $N - (\mu_{i}, 1.0)$ :

The function accepts three keyword arguments:

* **start** - int, the first mean in the sequence (inclusive)
* **end** - int, the last mean int the sequence (inclusive)
* **step** - int, the difference between mean i and mean i + 1.

For example, the following code creates a simulation model with 10 designs with means 1 to 10 and unit variance.

```python
model = guassian_sequence_model(1, 10)
```

To create a simulation model with 5 designs where $\mu_{i+1} - \mu_i = 2 $ :

```python
model = guassian_sequence_model(1, 10, step=2)
```

### Creating a custom model with known designs
Instead of a sequence of normal distributions with unit variance, it is possible to create a custom set of designs with varying variances. Use the `custom_guassian_model` function for this task.  For example to create a custom set of designs: 

```python
means = [5, 8, 1, 2, 1, 7]
variances = [0.1, 1.2, 1.4, 0.3, 0.8]

custom_model = custom_guassian_model(means, variances)
```

The following code demonstrates how to create a sequence of 100 designs with variances that are 10% of the mean. 

```python
n_designs = 100
means = [i for i in range(n_designs+1)]
variances = [j*0.1 for j in range(n_designs+1)]

custom_model = custom_guassian_model(means, variances)
```


### Creating a model with randomly sampled designs

To create a model with a set of unknown designs (within a specified mean and variance tolerance) use 

```python
mean_low = 1.0
mean_high = 15.0
var_low = 0.1
var_high = 2.0
n_designs = 15

model = random_guassian_model(mean_low, mean_high, var_low, var_high, n_designs)
```

Where:
* **mean_low** - float, a lower bound on the means of the output distributions
* **mean_high** - float, an upper bound on the means
* **var_low** - float, a lower bound on the variance of the output distributions
* **var_high** - float, an upper bound on the variances.
* **n_designs** - int, the number of designs to create.

### A manual optimisation framework

Before using the **Optimisation via Simulation** (OvS) procedures, it is recommended that you get a feel for the framework in which the OvS procedures operate.  To do this we will create some models and explore them using a `ManualOptimiser`.  This allows the user to run independent and multiple replications of the model yourself independent of any algorithm.  The `ManualOptimiser` keeps track of the means, variances and number of replications run for each design.

A `ManualOptimiser` object requires two parameters when it is created.  

* model - object, e.g. a model that is a sequence of normal distributions
* n_designs - the number of designs to be considered.

In [3]:
manual_opt = ManualOptimiser(model=guassian_sequence_model(1, 10),
                             n_designs=10)

* We can print the optimiser object to help us remember what parameters we set.

In [4]:
print(manual_opt)

ManualOptimiser(model=BanditCasino(), n_designs=10, verbose=False)


* Follow Law and Kelton's advice and run 3 initial replications of each design.

In [5]:
manual_opt.simulate_designs(replications=3)

The optimiser keeps track of the allocation of replications across each design.  For efficiency it doesn't store each individual observation, but it does compute a running mean and variance.

* Let's have a look at the replication allocation between and results of each design.

In [6]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [7]:
np.set_printoptions(precision=1) # this is a hack to view at 1 decimal place.
manual_opt.means

array([ 0.7,  2.1,  3.2,  3.6,  5.7,  6.8,  6.9,  8.8,  7.8, 11.1])

In [8]:
manual_opt.sigmas

array([3.5, 0.6, 0.9, 0.1, 0.3, 2.4, 1.2, 0.2, 0.7, 1. ])

Now let's run 1 additional replication of the top 3 designs. 
Note - in python arrays are **zero indexed**.  This means that design 1 has index 0.

In [ ]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=1)

In [ ]:
manual_opt.allocations

In [ ]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=2)

In [ ]:
manual_opt.means

In [ ]:
manual_opt.sigmas

### Manual Optimisation of a model with unknown means.

Now have a go yourself.  This time create a model with random designs.  Run as many replications of each design as you think is neccessary to make a decision abouit which design is best.  Here we define best as the design with the largest mean value.

In [ ]:
rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

manual_opt = ManualOptimiser(model=rnd_model, n_designs=10)

In [ ]:
#insert your code here...


## Procedure **KN**

The first Ranking and Selection (R&S) algorithm we will explore is **Procedure KN** developed by Kim and Nelson. This is an *Indifference Zone* (IZ) approach to R&S.  IZ procedures exploit the the idea that the performance of one or more of the sub-optimal designs are in fact so close to the best performing system that a user does not care which one is chosen (the decision maker is said to be *indifferent* to this choice).  To do this a user must specify a quantity $\delta$.  Procedure KN provides a theorectical guarantee to select the best system or a system within $\delta$ of the best with probability of $1 - \alpha$.

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [3]:
from ovs.indifference_zone import KN, KNPlusPlus

In [4]:
n_designs = 10
model = guassian_sequence_model(1, n_designs)

#model = random_guassian_model(mean_low=1, mean_high=15, 
#                              var_low=1, var_high=1,
#                              n_designs=10)

kn = KN(model=model, 
        n_designs=10, 
        delta=1.0, 
        alpha=0.05, 
        n_0=5)

In [5]:
model[model.best_design]._mu

10

In [6]:
#best design index
model.best_design

9

In [16]:
kn.reset()
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._allocations))
print('total reps\t{0}'.format(kn._allocations.sum()))
print('means\t\t{0}'.format(kn._means))
model[best_design[0]]._mu

best design	[9]
allocations	[ 5  5  5  5  5 10 25 10 16 26]
total reps	112
means		[ 1.27230125  2.5574599   3.3768002   4.4950446   4.44604417  5.96893095
  6.98494855  7.95448096  8.76947585 10.13364961]


10

In [32]:
knpp = KNPlusPlus(model=model, 
                  n_designs=10, 
                  delta=1.0, 
                  alpha=0.05, 
                  n_0=5)

In [33]:
best_design = knpp.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(knpp._allocations))
print('total reps\t{0}'.format(knpp._allocations.sum()))
print('means\t\t{0}'.format(knpp._means))
print('vars\t\t{0}'.format(knpp._vars))
model[best_design[0]]._mu

best design	[9]
allocations	[5 5 5 5 5 5 5 5 5 6]
total reps	51
means		[-0.48086413 -2.52236078 -3.45264768 -4.55354971 -5.1544865  -6.48816165
 -7.43699926 -8.00662423 -8.93729333 -9.5623595 ]
vars		[0.32646246 0.28613613 0.49533396 0.32714145 1.00774337 2.68299547
 0.41628835 0.69824525 0.46776161 0.96033603]


10

In [44]:
n_designs = 10
#means = [i for i in range(n_designs+1)]
#variances = [j*0.1 for j in range(n_designs+1)]

means = [4, 4.1, 4.2, 4, 4.1, 4.3, 4, 4.1, 4.2, 4.2]
variances = [1, 1, 1, 0.1, 0.1, 10, 10, 10, 10, 0.1]

guass_model = custom_guassian_model(means, variances)

kn = KN(model=guass_model, 
        n_designs=n_designs, 
        delta=0.15, 
        alpha=0.05, 
        n_0=50)

#print out which design indexes are in the indifference zone
np.where(4.3 - np.array(means) <= 0.15)[0]

array([2, 5, 8, 9])

In [45]:
kn.reset()
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._allocations))
print('total reps\t{0}'.format(kn._allocations.sum()))
print('means\t\t{0}'.format(kn._means))
model[best_design[0]]._mu

best design	[5]
allocations	[ 228  236 5661   50   50 9321 8661 9320 9320   50]
total reps	42897
means		[3.98355209 3.99089484 4.21053632 4.00427739 4.08636254 4.5359127
 4.16717206 4.15243085 4.25790498 4.17253833]


6

In [39]:
knpp = KNPlusPlus(model=guass_model, 
        n_designs=n_designs, 
        delta=0.15, 
        alpha=0.05, 
        n_0=5)

best_design = knpp.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(knpp._allocations))
print('total reps\t{0}'.format(knpp._allocations.sum()))
print('means\t\t{0}'.format(knpp._means))

best design	[5]
allocations	[5 5 5 5 5 6 5 5 5 5]
total reps	51
means		[-3.64155142 -4.07738943 -4.43214353 -4.04353978 -4.08437033 -8.78479154
 -4.95121291 -0.35734985 -1.86576327 -4.14704012]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [ ]:
from ovs.fixed_budget import OCBA, OCBAM

In [ ]:
n_designs = 10
guass_model = guassian_sequence_model(1, n_designs)

rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

ocba = OCBA(model=rnd_model, 
            n_designs=n_designs, 
            budget=500, 
            delta=5, 
            n_0=5, 
            obj='max')

In [ ]:
print(ocba)

call the `solve()` method to run the optimisation

In [ ]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

#### A harder problem...

In [ ]:
#try varying the budget, delta and n_0.  What is the effect?
#e.g try a budge

means = [4, 4.1, 4.2, 4, 4.1, 4.2, 4, 4.1, 4.2, 4.3]
variances = [1, 1, 1, 0.1, 0.1, 0.1, 10, 10, 10, 10]

guass_model = custom_guassian_model(means, variances)

ocba = OCBA(model=guass_model, 
            n_designs=n_designs, 
            budget=10000, 
            delta=5, 
            n_0=20, 
            obj='max')

In [ ]:
ocba.reset()
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))


np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))

## Optimal Computing Budget Allocation Top M (OCBA-m)

OCBA-m extended OCBA to identify the top m designs.  

An object of type `OCBAM` takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **m** - int, $m$ the number of top designs to return.  m must be >=2.  For m = 1 see OCBA.
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [ ]:
n_designs = 10
guass_model = guassian_sequence_model(1, n_designs+1)

ocbam = OCBAM(model=guass_model, 
              n_designs=n_designs, 
              budget=500, 
              delta=5, 
              n_0=5, 
              m=2,
              obj='max')

In [ ]:
print(ocbam)

In [ ]:
results = ocbam.solve()
print('best designs:\t{}'.format(np.sort(results)))
print('allocations:\t{}'.format(ocbam._allocations))
print('total reps:\t{}'.format(ocbam._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocbam._means))
print('vars:\t\t{0}'.format(ocbam._vars))
print('SEs:\t\t{0}'.format(ocbam._ses))

# Evaluation of the methods

In [ ]:
from ovs.evaluation import Experiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

In [ ]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

## Try different budgets

In [ ]:
from ovs.evaluation import GridExperiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [ ]:
results = exp.fit()

In [ ]:
results